In [1]:
from ch2.data import *
date = '2017-02-07 07:18:50'
s = session('-v5')
df = activity_statistics(s, DISTANCE, HEART_RATE, HR_ZONE, POWER_ESTIMATE, 
                         local_time=date, activity_group_name='Bike', with_timespan=True)
df.describe()

    INFO: Using database at /home/andrew/.ch2/database.sqlr
    INFO: Using Activity Journal ActivityJournal Bike 2017-02-07 10:18:50 to 2017-02-07 11:50:35


,Distance,Heart Rate,Power Estimate,HR Zone,Timespan ID
count,1183.000000,1180.000000,1183.000000,1180.000000,1172.000000
mean,16299.165528,112.094068,102.569651,2.136887,9152.275597
std,9292.581342,18.811294,146.161117,1.063538,12.065505
min,0.000000,69.000000,0.000000,1.000000,9093.000000
25%,8498.610000,95.000000,0.000000,1.000000,9154.000000
50%,16485.660000,111.000000,5.928585,2.271861,9155.000000
75%,24217.010000,132.000000,173.459564,3.246753,9155.000000
max,32054.430000,142.000000,628.054453,3.837072,9159.000000


In [2]:
import pandas as pd
import numpy as np

t4d = pd.DataFrame({TIME: df.index}, index=df[DISTANCE])
t4d = t4d[~t4d.index.duplicated(keep='last')]
t4d[TIME] = (t4d[TIME] - t4d[TIME].iloc[0]).astype(np.int64) / 1e9
t4d.describe()
lo, hi = t4d.index.min(), t4d.index.max()
delta = 10
lo, hi = int(lo / delta) * delta, (1 + int(hi / delta)) * delta
ed = pd.DataFrame({'keep': True}, index=np.arange(lo, hi, delta))
et4d = t4d.join(ed, how='outer', sort=True)
et4d.interpolate(method='slinear', limit_area='inside', inplace=True)
et4d = et4d.loc[et4d['keep'] == True].drop(columns=['keep'])
print(et4d.describe())

              Time
count  3206.000000
mean   2926.848337
std    1488.315728
min       0.000000
25%    1725.486092
50%    2998.738791
75%    4215.272857
max    5488.900474


In [3]:
d = 10  # km
n = d * 1000 / delta  # bins
print(et4d.diff(periods=n))

                Time
0.0              NaN
10.0             NaN
20.0             NaN
30.0             NaN
40.0             NaN
50.0             NaN
60.0             NaN
70.0             NaN
80.0             NaN
90.0             NaN
100.0            NaN
110.0            NaN
120.0            NaN
130.0            NaN
140.0            NaN
150.0            NaN
160.0            NaN
170.0            NaN
180.0            NaN
190.0            NaN
200.0            NaN
210.0            NaN
220.0            NaN
230.0            NaN
240.0            NaN
250.0            NaN
260.0            NaN
270.0            NaN
280.0            NaN
290.0            NaN
...              ...
31760.0  1566.151223
31770.0  1566.364180
31780.0  1566.501459
31790.0  1566.549431
31800.0  1566.684205
31810.0  1566.881720
31820.0  1567.205588
31830.0  1567.542565
31840.0  1568.034603
31850.0  1568.533102
31860.0  1569.031601
31870.0  1569.530100
31880.0  1570.135551
31890.0  1570.844937
31900.0  1571.446962
31910.0  1571

In [4]:
from ch2.data.activity import *

print(times_for_distance(df))

   DEBUG: Linear resample with type <class 'pandas.core.indexes.numeric.Float64Index'>, columns Index(['Time'], dtype='object')


{'Min 5km Time': 626.5516572078723, 'Med 5km Time': 768.7816192945772, 'Min 10km Time': 1388.198709207476, 'Med 10km Time': 1561.0595238557944, 'Min 15km Time': 2162.105926772734, 'Med 15km Time': 2330.5670497458195, 'Min 20km Time': 2969.74279720213, 'Med 20km Time': 3094.506905951884, 'Min 25km Time': 3709.8855713420926, 'Med 25km Time': 3996.6095037844943}


In [5]:
from ch2.data.heart_rate import *
from ch2.squeal import *
from ch2.lib.date import *

# upper limits
zones = hr_zones_from_database(s, s.query(ActivityGroup).filter(ActivityGroup.name == 'Bike').one(), 
                               local_time_to_time(date))
print(zones)

[104.72, 127.82, 144.76, 161.7, 186.34, 1538.46]


In [6]:
ldf = linear_resample_time(df)
hrz = pd.cut(ldf[HR_ZONE], bins=range(7)).value_counts()
for x in hrz.iteritems():
    print(x)

   DEBUG: Linear resample with type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>, columns Index(['Distance', 'Heart Rate', 'Power Estimate', 'HR Zone', 'Timespan ID'], dtype='object')


(Interval(0, 1, closed='right'), 662)
(Interval(3, 4, closed='right'), 613)
(Interval(2, 3, closed='right'), 429)
(Interval(1, 2, closed='right'), 26)
(Interval(5, 6, closed='right'), 0)
(Interval(4, 5, closed='right'), 0)


In [7]:
print(hrz_stats(df))

   DEBUG: Linear resample with type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>, columns Index(['Distance', 'Heart Rate', 'Power Estimate', 'HR Zone', 'Timespan ID'], dtype='object')


{'Percent in Z1': 38.26589595375722, 'Time in Z1': 1986.0, 'Percent in Z4': 35.433526011560694, 'Time in Z4': 1839.0, 'Percent in Z3': 24.797687861271676, 'Time in Z3': 1287.0, 'Percent in Z2': 1.5028901734104045, 'Time in Z2': 78.0, 'Percent in Z6': 0.0, 'Time in Z6': 0.0, 'Percent in Z5': 0.0, 'Time in Z5': 0.0}


In [8]:
print(max_mean(df))

   DEBUG: Linear resample with type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>, columns Index(['Distance', 'Heart Rate', 'Power Estimate', 'HR Zone', 'Timespan ID'], dtype='object')


{'Max Med EP 5m': nan, 'Max Med EP 10m': 612.3218920869258, 'Max Med EP 30m': 422.4286949119414, 'Max Med EP 60m': 252.29633772000184, 'Max Med EP 90m': 242.2063112390982, 'Max Med EP 120m': 212.05775709298328, 'Max Med EP 180m': 200.92245669369575}


/home/andrew/project/ch2/choochoo/ch2/data/activity.py:60: RuntimeWarning: invalid value encountered in double_scalars
  stats[template % target] = diff.max() / n


In [9]:
print(max_med(df))

   DEBUG: Linear resample with type <class 'pandas.core.indexes.datetimes.DatetimeIndex'>, columns Index(['Distance', 'Heart Rate', 'Power Estimate', 'HR Zone', 'Timespan ID'], dtype='object')
   DEBUG: Largest gap is 90.0s
   DEBUG: Target 5m is 30 samples (delta 10s)
   DEBUG: Split data into 9 sections for 5m with max gap of 15.0s
   DEBUG: Target 10m is 60 samples (delta 10s)
   DEBUG: Split data into 9 sections for 10m with max gap of 15.0s
   DEBUG: Target 30m is 180 samples (delta 10s)
   DEBUG: Split data into 9 sections for 30m with max gap of 18.0s
   DEBUG: Target 60m is 360 samples (delta 10s)
   DEBUG: Split data into 6 sections for 60m with max gap of 36.0s
   DEBUG: Target 90m is 540 samples (delta 10s)
   DEBUG: Split data into 3 sections for 90m with max gap of 54.0s
   DEBUG: Target 120m is 720 samples (delta 10s)
   DEBUG: Split data into 3 sections for 120m with max gap of 72.0s
   DEBUG: Target 180m is 1080 samples (delta 10s)
   DEBUG: Split data into 1 sections f

{'Max Med HR 5m': 132.25, 'Max Med HR 10m': 131.91666666666669, 'Max Med HR 30m': 126.23611111111111}
